In [1]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/archive (4).zip'

Archive:  /content/drive/MyDrive/archive (4).zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: tes

In [4]:
train_path = "/content/train_data"
test_path = "/content/test_data"

In [5]:
train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255)

In [6]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 150 images belonging to 1 classes.
Found 157 images belonging to 1 classes.


VGG16


In [7]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [8]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 2s 0us/step


In [9]:
# Train model with existing weights

for layer in vgg.layers:
  print(layer)

In [10]:
# Train model with existing weights

for layer in vgg.layers:
  layer.trainable=False

In [11]:
x = Flatten()(vgg.output)

In [12]:

# output layer

prediction = Dense(4,activation='softmax')(x)

In [13]:
# Create Vgg16 model

model = Model(inputs=vgg.input,outputs=prediction)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
     

In [16]:
model.fit_generator(train,validation_data=test,epochs=4,steps_per_epoch=len(train),
                    validation_steps=len(test))

<ipython-input-16-0aa3deeaa555>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=test,epochs=4,steps_per_epoch=len(train),


Epoch 1/4
7/7 [==============================] - 109s 15s/step - loss: 5.7719 - accuracy: 0.5867 - val_loss: 5.8155 - val_accuracy: 0.5669
Epoch 2/4
7/7 [==============================] - 91s 14s/step - loss: 5.8019 - accuracy: 0.5400 - val_loss: 5.7910 - val_accuracy: 0.4904
Epoch 3/4
7/7 [==============================] - 87s 14s/step - loss: 5.7862 - accuracy: 0.4800 - val_loss: 5.8008 - val_accuracy: 0.5541
Epoch 4/4
7/7 [==============================] - 89s 14s/step - loss: 5.7978 - accuracy: 0.6067 - val_loss: 5.8114 - val_accuracy: 0.6051


ResNet50

In [18]:
from tensorflow.keras.applications.resnet50 import ResNet50
     

resnet = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 3s 0us/step


In [19]:
for layer in resnet.layers:
  layer.trainable=False

In [20]:
for layer in resnet.layers:
  print(layer)

In [21]:
x = Flatten()(resnet.output)
     

out = Dense(4, activation='softmax')(x)

In [22]:
res_model = Model(inputs=resnet.input,outputs=out)

In [23]:
res_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [24]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
res_model.fit(train,epochs=5,validation_data=test,steps_per_epoch=len(train),
              validation_steps=len(test))

Epoch 1/5
7/7 [==============================] - 107s 16s/step - loss: 6.9812 - accuracy: 0.0000e+00 - val_loss: 7.0464 - val_accuracy: 0.0000e+00
Epoch 2/5
7/7 [==============================] - 93s 14s/step - loss: 6.9941 - accuracy: 0.0000e+00 - val_loss: 7.0370 - val_accuracy: 0.0000e+00
Epoch 3/5
7/7 [==============================] - 122s 19s/step - loss: 6.9980 - accuracy: 0.0000e+00 - val_loss: 7.0184 - val_accuracy: 0.0000e+00
Epoch 4/5
7/7 [==============================] - 121s 19s/step - loss: 6.9982 - accuracy: 0.0000e+00 - val_loss: 6.9395 - val_accuracy: 0.0000e+00
Epoch 5/5
7/7 [==============================] - 91s 15s/step - loss: 6.9967 - accuracy: 0.0000e+00 - val_loss: 7.0209 - val_accuracy: 0.0000e+00
